In [3]:
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import datetime
from datetime import date 
import time
import json
import csv
import numpy as np
import re
import lxml.html
import os
import tabula

In [4]:
wta_path = "https://wtafiles.wtatennis.com/pdf/rankings/All_YTD_Prize_Money.pdf"

In [5]:
atp_path = "https://www.protennislive.com/posting/ramr/current_prize.pdf"

In [6]:
wta_dfs = tabula.read_pdf(wta_path, stream=False, user_agent="Mozilla/5.0", pages='1-11')
# read_pdf returns list of DataFrames
print(len(wta_dfs))

11


In [7]:
print(wta_dfs[2])

    Standing                 Unnamed: 0  Name  Nat  Singles  Doubles    Mixed  \
0         93               BUREL, CLARA   NaN  FRA  479,101   20,370    5,000   
1         94              MINNEN, GREET   NaN  BEL  390,947  107,891      NaN   
2         95              NAVARRO, EMMA   NaN  USA  463,806   34,283      NaN   
3         96       PARRIZAS DIAZ, NURIA   NaN  ESP  486,764    9,125      NaN   
4         97            VOLYNETS, KATIE   NaN  USA  492,258      697      NaN   
5         98               GALFI, DALMA   NaN  HUN  459,858   26,841      NaN   
6         99           KALINSKAYA, ANNA   NaN  NaN  386,918   96,583      NaN   
7        100            MARINO, REBECCA   NaN  CAN  455,476   25,081      NaN   
8        101                JUVAN, KAJA   NaN  SLO  479,100      435      NaN   
9        102           BAINDL, KATERYNA   NaN  UKR  464,741   13,929      NaN   
10       103               KANEPI, KAIA   NaN  EST  462,316   13,374      NaN   
11       104           BRENG

In [8]:
#print(wta_dfs[2].columns)

In [9]:
#print(wta_dfs[10])

In [10]:
#DO NOT USE, WILL ERROR: 
#wta_df.drop(columns=['Name'], inplace=True)

In [11]:
#Faulty original failed to specify that "name" was a column 
# if (len(wta_df.columns) == 7):
#     wta_df.drop('Name', inplace=True)
#     wta_df.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)

In [ ]:
## TO DO: concat AFTER changing capitalization under "name" 

In [12]:
#only keep the columns that are common to the two dfs
#wta_dfs = wta_dfs[['Name', , ]
women_df= pd.DataFrame([])

for wta_df in wta_dfs:
    if (len(wta_df.columns) == 8):
        wta_df.drop(columns=['Name'], inplace=True)
        wta_df.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)
    wta_df.rename(columns={'Standing': 'Rank'}, inplace=True)   
    wta_df = wta_df[['Rank','Name', 'Total', 'Singles','Doubles' ]]
    women_df = pd.concat([women_df, wta_df])
    


In [13]:
print(women_df.columns)

Index(['Rank', 'Name', 'Total', 'Singles', 'Doubles'], dtype='object')


In [14]:
#previously: didn't have the INPLACE parameter 

In [15]:
women_df.reset_index(drop = True, inplace=True)

In [16]:
#Cutting off at index 499
women_df = women_df[women_df['Rank'] <= 500]

In [17]:
#len(women_df)

In [18]:
#women_df.to_csv("files/2023-10-09_wta.csv")

In [19]:
#some columns have missing player names - pages == resolved thanks to RENAME 

In [20]:
atp_dfs = tabula.read_pdf(atp_path, stream=True, user_agent="Mozilla/5.0", pages='1-12', pandas_options={'header': None})

In [21]:
#print(atp_dfs[0].columns)

In [22]:
#print(atp_dfs[0])

In [23]:
men_df= pd.DataFrame([])

for atp_df in atp_dfs:
    atp_df.columns = ['Rank', 'Name', 'Total', 'Singles','Doubles', 'Career']
    atp_df = atp_df[['Rank','Name', 'Total', 'Singles','Doubles' ]]
    men_df = pd.concat([men_df, atp_df])

In [24]:
men_df['Total'] = men_df['Total'].str.strip('$')

In [25]:
men_df['Singles'] = men_df['Singles'].str.strip('$')

In [26]:
men_df['Doubles'] = men_df['Doubles'].str.strip('$')

In [27]:
#NOT WORKING: men_df["Rank"] = men_df["Rank"].astype(int)

In [28]:
#print(men_df.columns)

In [29]:
men_df.reset_index(drop = True, inplace=True)

In [30]:
#WE JUST LOST DJOKOVIC!!!!  --> resolved with correct tabula formula 

In [31]:
#NOT WORKING, either as int or as string 
#men_df = men_df[men_df['Rank'] <= 500]

In [32]:
#we used the index 
men_df = men_df[men_df.index <= 499]

In [33]:
#len(men_df)

In [34]:
#mkdir files 
#cd files 
#name each file with the date  
#also store all weekly rank data in a single file, for atp and wta ("all_weeks")

In [35]:
#already done:
#os.mkdir("files")

In [36]:
date = date.today()

In [37]:
print(date)

2023-11-03


In [38]:
women_df.to_csv(f"files/{date}_wta.csv")

In [39]:
#PROBLEM: the index vaues repeat themselves instead of being sequential == resolved via "inplace=true
#LETS SEE AFTER WE COMBINE 

In [40]:
men_df.to_csv(f"files/{date}_atp.csv")

In [41]:
men_df['gender'] = 'male'
women_df['gender'] = 'female'

In [42]:
mixed_df = pd.concat([women_df, men_df])

In [43]:
#print(mixed_df)

In [44]:
#print(mixed_df.iloc[55])

In [45]:
#print(mixed_df.iloc[552])

In [46]:
#problems with women_df from last session: 
# - ranks not in sequence: 
# - no names for some pages: 
# - doubles earnings missing last 3 digits:

In [47]:
#mixed_df.reset_index(inplace=True)
mixed_df.reset_index(drop = True, inplace=True)

In [48]:
#******** RETURN TO THIS AFTER 

In [49]:
#mixed_df.to_csv(f"files/{date}_mixed.csv")

In [50]:
#HOW/WHAT do we compare??
#compare [gender]=women at RANK i vs gender - men at rank i
#Do we keep the names or just the ranks? 
# what does DW show? 
#OPTION 1
#weekly updated table of the difference between W1 M1
#BUT if you hover over any of the rows, you get the info on absolute earnings and player name ()both are bound to change due to ranking change) 
#OPTION 2
#Show difference at each rank, plus respective names??? CTA -- hover to see actual earnings$$ 

#create a new df called "difference" loop thru mixed df by looping with gender-female from rank 1 to 500, 
# then gender-male at rank value 1 to 500, 


In [51]:
# for player in mixed_df:
#     if mixed_df[[gender = female && index ]]

In [52]:
# diff = 0
# for woman in women_df: 
#     difference_df = pd.DataFrame()


    
    

In [53]:
# ranks = []
# diff = []

# for index in women_df.index:
#     men_total = men_df.iloc[index]['Total']
#     women_total = women_df.iloc[index]['Total']
#     ranks.append(index + 1)
#     diff.append(men_total - women_total)
    

In [54]:
#remove commas 

In [55]:
# men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
# women_total = int(women_df.iloc[index]['Total'].replace(',', ''))

In [56]:
# ranks = []
# diff = []

# for index in women_df.index:
#     men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
#     women_total = int(women_df.iloc[index]['Total'].replace(',', ''))
#     ranks.append(index + 1)
#     diff.append(men_total - women_total)

In [57]:
# len(diff)

In [58]:
# diff_df = pd.DataFrame([])
# diff_df['Rank'] = ranks
# diff_df['diff'] = diff

In [59]:
# diff_df.head()

In [60]:
#next: add names?? 

In [61]:
# diff_df = pd.DataFrame([])
# diff_df['Rank'] = ranks
# diff_df['diff'] = diff
# diff_df['women'] = women_df['Name']
# diff_df['men'] = men_df['Name']

In [62]:
# diff_df.head(50)

In [63]:
#NEXT: percentage difference 

In [64]:
#USE THE BELOW: 
ranks = []
diff = []
pct = []

for index in women_df.index:
    men_total = int(men_df.iloc[index]['Total'].replace(',', ''))
    women_total = int(women_df.iloc[index]['Total'].replace(',', ''))
    ranks.append(index + 1)
    diff.append(men_total - women_total)
    pct.append((women_total/men_total)*100)

In [65]:
diff_df = pd.DataFrame([])
diff_df['Rank'] = ranks
diff_df['diff'] = diff
diff_df['pct'] = pct
diff_df['women'] = women_df['Name']
diff_df['men'] = men_df['Name']

In [66]:
#diff_df.head(20)

In [67]:
#diff_df.tail(30)

In [68]:
#SAVE THIS DIFF dataframe! --> feed to DW 

In [69]:
diff_df.to_csv(f"files/{date}_diff.csv")

In [70]:
#ALSO SAVE TO UNDATED csv!!! 

In [71]:
diff_df.to_csv("latest_diff.csv")

In [72]:
ls 

OCT9.ipynb                latest_diff.csv           pay_gap.ipynb
README.md                 old_atp_live_ranking.csv  tennisOCT3.ipynb
atp_live_ranking.csv      old_tennispay.ipynb       tennisgpt.ipynb
files/                    old_tennisweekly.ipynb    weekly_race_to_top.csv


In [74]:
ls-la

total 6448
drwxr-xr-x  18 mfhan  staff      576 Oct 30 16:14 ./
drwxr-xr-x  10 mfhan  staff      320 Oct 24 19:29 ../
-rw-r--r--@  1 mfhan  staff    10244 Oct 30 15:23 .DS_Store
drwxr-xr-x  15 mfhan  staff      480 Oct 30 16:14 .git/
drwxr-xr-x   4 mfhan  staff      128 Oct 22 22:20 .github/
drwxr-xr-x  12 mfhan  staff      384 Oct 22 22:21 .ipynb_checkpoints/
-rw-r--r--   1 mfhan  staff    87443 Oct  9 19:05 OCT9.ipynb
-rw-r--r--   1 mfhan  staff       20 Oct 22 20:26 README.md
-rw-r--r--@  1 mfhan  staff       47 Sep 21 17:11 atp_live_ranking.csv
drwxr-xr-x  25 mfhan  staff      800 Nov  3 14:01 files/
-rw-r--r--@  1 mfhan  staff    34473 Nov  3 14:01 latest_diff.csv
-rw-r--r--@  1 mfhan  staff       20 Sep 21 17:03 old_atp_live_ranking.csv
-rw-r--r--   1 mfhan  staff    11970 Oct  9 14:40 old_tennispay.ipynb
-rw-r--r--   1 mfhan  staff  1565449 Oct  3 16:06 old_tennisweekly.ipynb
-rw-r--r--   1 mfhan  staff    66770 Oct 23 15:26 pay_gap.ipynb
-rw-r--r--   1 mfhan  staff   813332 Oct